## 1. 处理缺失数据

### 滤除缺失数据
`dropna` 
- 对于Series，返回一个仅含非空数据和索引值的Series
- 对于DataFrame，默认返回一个丢弃了任何含有缺失值的行的DataFrame  
  
参数：  
`how='all'` 只丢弃全为NA的行  
`axis=1` 丢弃列  
`thresh=n` n为整数，保留至少有n个非NaN数据的行/列  

In [1]:
import numpy as np
import pandas as pd
from numpy import nan as NA

In [2]:
data = pd.Series([1, NA, 4, NA, 9])  # 对于Series的dropna操作
data.dropna()  # 等价于 data[data.notnull()]

0    1.0
2    4.0
4    9.0
dtype: float64

In [3]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [4]:
data.dropna()  # 只保留没有NA的行

,0,1,2
0,1.0,6.5,3.0


In [5]:
data.dropna(how='all')  # 只丢弃全为NA的行

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [6]:
data[3] = NA
data

,0,1,2,3
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [7]:
data.dropna(axis=1, how='all')  # 只丢弃全为NA的列

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
# 保留至少有2个非NaN数据的行，其中1行非NaN数据为1个，2行为0个，因此这两行不不保留，而2行和3行非NaN数据为3个和2个，均给与保留
data.dropna(thresh=2)

,0,1,2,3
0,1.0,6.5,3.0,NaN
3,NaN,6.5,3.0,NaN


### 填充缺失数据
`fillna`
- 通过常数调用该函数，就会将缺失值替换为那个常数
- 通过一个字典调用该函数，可以实现对不同列填充不同的值，若字典的列不存在则丢弃
- 默认返回一个新对象
  
参数  
`axis`：填充方向，默认0/'index'（对某列中的行进行填充，行方向），可选1/'columns'  
`method`：填充方式，默认'ffill'向前填充，可选'bfill'向后填充，使用方法类似于`reindex`的`method`  
`limit`：可以连续填充的最大数量，**注意是连续填充，并非单行/列可填充的最大数量**  
`inplace`：是否直接修改对象而不创建副本，默认'False'

In [9]:
data = pd.DataFrame(np.random.randint(50, size=35).reshape(
    7, 5), index=list('abcdefg'), columns=list('ABCDE'))
data.loc['c':'e', 'B'], data.loc['b':'d', 'C':'D'] = NA, NA
data.loc['b':'c', 'A'], data.loc['e':'g', 'A'] = NA, NA
data

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47
b,NaN,12.0,NaN,NaN,17
c,NaN,NaN,NaN,NaN,34
d,25.0,NaN,NaN,NaN,42
e,NaN,NaN,34.0,20.0,32
f,NaN,7.0,13.0,27.0,1
g,NaN,11.0,7.0,2.0,8


In [10]:
data.fillna(0)

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47
b,0.0,12.0,0.0,0.0,17
c,0.0,0.0,0.0,0.0,34
d,25.0,0.0,0.0,0.0,42
e,0.0,0.0,34.0,20.0,32
f,0.0,7.0,13.0,27.0,1
g,0.0,11.0,7.0,2.0,8


In [11]:
data.fillna({'B': 99, 'D': 55, 'X': 88})  # 使用字典填充，丢弃不存在的列

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47
b,NaN,12.0,NaN,55.0,17
c,NaN,99.0,NaN,55.0,34
d,25.0,99.0,NaN,55.0,42
e,NaN,99.0,34.0,20.0,32
f,NaN,7.0,13.0,27.0,1
g,NaN,11.0,7.0,2.0,8


In [12]:
# 向前填充，最大连续填充数量为2
# 注意A列用a行和d行的数值分别向前填充了2项，而并非整个A列最多填充2项
data.fillna(method='ffill', limit=2)

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47
b,28.0,12.0,4.0,18.0,17
c,28.0,12.0,4.0,18.0,34
d,25.0,12.0,NaN,NaN,42
e,25.0,NaN,34.0,20.0,32
f,25.0,7.0,13.0,27.0,1
g,NaN,11.0,7.0,2.0,8


In [13]:
# 向前填充，最大连续填充数量为2
# 注意A列用a行和d行的数值分别向前填充了2项，而并非整个A列最多填充2项
data.fillna(method='ffill', limit=2)

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47
b,28.0,12.0,4.0,18.0,17
c,28.0,12.0,4.0,18.0,34
d,25.0,12.0,NaN,NaN,42
e,25.0,NaN,34.0,20.0,32
f,25.0,7.0,13.0,27.0,1
g,NaN,11.0,7.0,2.0,8


In [14]:
# 对列方向进行向后填充，最大连续填充数量为2
data.fillna(axis=1, method='bfill', limit=2)

,A,B,C,D,E
a,28.0,4.0,4.0,18.0,47.0
b,12.0,12.0,17.0,17.0,17.0
c,NaN,NaN,34.0,34.0,34.0
d,25.0,NaN,42.0,42.0,42.0
e,34.0,34.0,34.0,20.0,32.0
f,7.0,7.0,13.0,27.0,1.0
g,11.0,11.0,7.0,2.0,8.0


## 2. 数据转换

### 移除重复数据
`duplicated`：返回一个布尔型Series，表示各行是否是重复行  
`drop_duplicates`：返回一个移除了重复行的DataFrame  
  
参数  
`subset`：传入list格式的列名，对指定列的内容进行重复项操作  
`keep`：默认'first'，移除重复值时保留第一个值；可选'last'，保留最后一个重复值  
`inplace`：是否直接修改对象而不创建副本，默认'False'

In [15]:
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1, 1, 2, 3, 3, 4, 4],
    'k3': list('aaabbbb')
})
data

,k1,k2,k3
0,one,1,a
1,two,1,a
2,one,2,a
3,two,3,b
4,one,3,b
5,two,4,b
6,two,4,b


In [16]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [17]:
data.drop_duplicates(['k1', 'k3'])  # 对指定列的内容进行重复项操作

,k1,k2,k3
0,one,1,a
1,two,1,a
3,two,3,b
4,one,3,b


In [18]:
data.drop_duplicates(['k1', 'k3'], keep='last')  # keep='last' 保留最后一个重复值

,k1,k2,k3
1,two,1,a
2,one,2,a
4,one,3,b
6,two,4,b


### 利用函数或映射进行数据转换
使用Series的`map()`函数，可以将映射对象的内容转换并填入DataFrame中。
1. 从DataFrame中取出需要映射的列得到Series
2. 然后该Series将映射对象代入`map()`函数中，得到由映射内容组成的Series
3. 最后将得到的Series添加至原DataFrame里。
  
> `Series.map()`用法：对象使用map代入一个函数/字典/Series  
1. 如果代入的是函数，将调用map的对象本身代入函数进行运算
2. 如果代入的是字典或Series，则将原对象里对应的值替换为字典或Series中该值作为key所对应的值；如原对象`d=pd.Series({'a':'z','b':'y'})`,`d.map({'z':1,'y':2})`,则d中的'x'和'y'被替换为了1和2

In [19]:
data = pd.DataFrame([list('dsafdfa'), [8, 9, 7, 7, 9, 7, 8]],
                    index=['ID', 'Score']).T
data

,ID,Score
0,d,8
1,s,9
2,a,7
3,f,7
4,d,9
5,f,7
6,a,8


In [20]:
d = {
    'a': 'Class-A',
    's': 'Class-S',
    'd': 'Class-D',
    'f': 'Class-F'
}  # 映射对象，ID->Class

# 以下3行代码等价于此代码：data['Class']=data['ID'].map(d)
ID = data['ID']  # 从DataFrame中取出需要映射的列得到Series
# 将映射对象ID代入map()函数中，并传入字典d，ID和字典d进行映射，得到ID值相对应d中的内容，并组成的Series返回
Class = ID.map(d)
data['Class'] = Class  # 将得到的Series添加至原DataFrame

data

,ID,Score,Class
0,d,8,Class-D
1,s,9,Class-S
2,a,7,Class-A
3,f,7,Class-F
4,d,9,Class-D
5,f,7,Class-F
6,a,8,Class-A


### 替换值
`replace`  
- 产生的是一个新的对象，除非传入`inplace=True`
- 一次性替换多个值，可以传入待替换值组成的列表及一个替换值
- 将不同的待替换值替换为不同的值，可以传入一个替换值的列表，或者传入一个字典
> 注意：data.replace方法与data.str.replace不同


In [21]:
data = pd.DataFrame(np.random.randint(-2, 2, size=25).reshape(5, 5))
data

,0,1,2,3,4
0,0,1,-1,-1,0
1,0,0,0,-2,0
2,-2,0,-1,-1,0
3,-1,0,-1,-2,-2
4,-2,-2,0,-1,1


In [22]:
data.replace(-1, NA)

,0,1,2,3,4
0,0.0,1,NaN,NaN,0
1,0.0,0,0.0,-2.0,0
2,-2.0,0,NaN,NaN,0
3,NaN,0,NaN,-2.0,-2
4,-2.0,-2,0.0,NaN,1


In [23]:
data.replace([-1, -2], NA)  # 一次替换2个值

,0,1,2,3,4
0,0.0,1.0,NaN,NaN,0.0
1,0.0,0.0,0.0,NaN,0.0
2,NaN,0.0,NaN,NaN,0.0
3,NaN,0.0,NaN,NaN,NaN
4,NaN,NaN,0.0,NaN,1.0


In [24]:
data.replace([-1, -2], ['x', 'y'])  # 将2个值替换为不同的2个替换值

,0,1,2,3,4
0,0,1,x,x,0
1,0,0,0,y,0
2,y,0,x,x,0
3,x,0,x,y,y
4,y,y,0,x,1


In [25]:
data.replace({-1: 'x', -2: 'y'})  # 使用字典实现替换

,0,1,2,3,4
0,0,1,x,x,0
1,0,0,0,y,0
2,y,0,x,x,0
3,x,0,x,y,y
4,y,y,0,x,1


### 重命名轴索引
- `map`：可以直接在原轴基础上进行修改，并将修改结果替换原轴  
  
  
- `rename`：对轴进行改名  
参数  
`index` `columns`：传入函数或者字典，用轴的值代入函数运算得到的结果，或轴的值对应的字典值修改轴的值
`inplace`：直接修改不生成副本

In [26]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=list('xyz'),
                    columns=list('ABCD'))
data

,A,B,C,D
x,0,1,2,3
y,4,5,6,7
z,8,9,10,11


In [27]:
# map代入匿名函数，将data.index的每个元素代入函数运算并将结果返回为新的Series
data.index.map(lambda x: 'Class-'+x.upper())

Index(['Class-X', 'Class-Y', 'Class-Z'], dtype='object')

In [28]:
# map代入字典，将data.index的每个元素替换为该元素作为key所对应的字典中的值，并返回
data.index.map({'x': 'Class-X', 'y': 'Class-Y', 'z': 'Class-Z'})

Index(['Class-X', 'Class-Y', 'Class-Z'], dtype='object')

In [29]:
# 在原index轴基础上修改index轴的内容，并用结果替换原index轴
data.index = data.index.map(lambda x: 'Class-'+x.upper())
data

,A,B,C,D
Class-X,0,1,2,3
Class-Y,4,5,6,7
Class-Z,8,9,10,11


In [30]:
# 使用rename修改轴，index使用字典对应的值，columns使用函数运算结果
data.rename(index={'Class-X': '班级X', 'Class-Y': '班级Y',
                   'Class-Z': '班级Z'}, columns=lambda x: x.lower())

,a,b,c,d
班级X,0,1,2,3
班级Y,4,5,6,7
班级Z,8,9,10,11
